In [2]:
import numpy as np
import pandas as pd
from shapely.geometry import Point
import geopandas as gpd


In [3]:
def find_census_tracts(lat, lon, census_tracts_gdf):
    """
    Identify the census tract containing the point and census tracts within 0.25 miles.
    """
    point = Point(lon, lat)
    
    # Ensure the census tracts are in WGS 84 (EPSG:4326) for point lookup
    census_tracts_gdf = census_tracts_gdf.to_crs(epsg=4326)
    actual_tract = census_tracts_gdf[census_tracts_gdf.contains(point)]
    
    # Convert to a projected CRS (meters) for distance calculations
    census_tracts_meters = census_tracts_gdf.to_crs(epsg=3857)
    point_meters = gpd.GeoSeries([point], crs=4326).to_crs(epsg=3857).iloc[0]
    
    # Buffer 0.25 miles (~402 meters) and find nearby census tracts
    point_buffer = point_meters.buffer(402)  # Distance in meters
    nearby_tracts = census_tracts_meters[census_tracts_meters.intersects(point_buffer)]
    
    tract_dict = {}
    if not actual_tract.empty:
        tract_dict["actual"] = actual_tract.iloc[0]["GEOID"]  # Assuming GEOID as census tract identifier
    
    for idx, row in nearby_tracts.iterrows():
        if row["GEOID"] != tract_dict.get("actual"):
            tract_dict[f"tract{idx}"] = row["GEOID"]
    
    return tract_dict

In [4]:
# Load Georgia boundary shapefile (adjust path if needed)
ga_boundary = gpd.read_file("../../data/raw/shapefiles/tl_2024_13_tract/tl_2024_13_tract.shp").to_crs(epsg=4326)



# Define grid spacing (degrees) - Adjust for more or fewer points
lat_min, lat_max = 30.3, 35.0  # Approximate latitude range for Georgia
lon_min, lon_max = -85.7, -80.8  # Approximate longitude range for Georgia
grid_spacing = 0.1  # Adjust this for higher/lower resolution

# Generate grid points
lat_values = np.arange(lat_min, lat_max, grid_spacing)
lon_values = np.arange(lon_min, lon_max, grid_spacing)
grid_points = [(lat, lon) for lat in lat_values for lon in lon_values]

# Convert to GeoDataFrame and filter only points within Georgia
grid_gdf = gpd.GeoDataFrame(geometry=[Point(lon, lat) for lat, lon in grid_points], crs="EPSG:4326")
grid_gdf = grid_gdf[grid_gdf.within(ga_boundary.unary_union)]  # Keep only points inside Georgia

# Calculate Stable Communities Score for each grid point
results = []
for point in grid_gdf.geometry:
    lat, lon = point.y, point.x
    tract_info = find_census_tracts(lat, lon, gdf)
    score_info = calculate_indicators_score(tract_info, indicators_df)

    results.append({"latitude": lat, "longitude": lon, "score": score_info["score"]})

# Save as CSV for QGIS
df = pd.DataFrame(results)
df.to_csv("../../data/processed/scoring_indicators/georgia_stable_communities_grid.csv", index=False)

/var/folders/w2/jx7kgvv14cx4km6tl2rymsmw0000gn/T/ipykernel_39690/3568557377.py:18: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  grid_gdf = grid_gdf[grid_gdf.within(ga_boundary.unary_union)]  # Keep only points inside Georgia


NameError: name 'gdf' is not defined